In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill
import submatcher as sm
import hashlib
import re

In [2]:
# inp = ["flour","butter","milk", "egg", "cheese"]
inp = {"Egg": "egg", "Flour": "Flour", "Milk": "Milk", "Salt": "Salt", "Sugar": "Sugar", "Vanilla Extract": "Vanilla Extract"}

In [3]:
allr = rr.GetRecipies(inp, recnum = 9)


Recipe cache found
recipe Easy Blonde Brownies cache found
recipe Skinny Low Fat Chocolate Cake Roundup cache found
recipe Zucchini Waffles cache found
recipe Chocolate Self-Saucing Pudding cache found
recipe French Toast Maple Sausage Muffins cache found
recipe Blueberry Sourdough Pancakes/with sourdough starter cache found
recipe Simple Swedish Pancakes with Bacon Roll-Ups cache found
recipe Grandma’s Muffin s cache found
recipe Liege Waffles With Nutella Spread cache found


In [4]:
allr.to_json()

{'Easy Blonde Brownies': {'attrs': {'vegetarian': True,
   'vegan': False,
   'glutenFree': False,
   'dairyFree': False},
  'c': True,
  'cuisines': ['American'],
  'id': 525788,
  'imglink': 'https://spoonacular.com/recipeImages/525788-312x231.jpg',
  'missedCount': 2,
  'missedIngs': ['baking powder', 'butter'],
  'name': 'Easy Blonde Brownies',
  'price': 12.22,
  'serving': 16,
  'subIngs': {'baking powder': {'baking powder': ['baking soda plus cream of tartar ',
     ' baking soda plus buttermilk (decrease liquid in recipe by ']},
   'butter': {'butter (salted)': ['margarine ',
     ' shortening plus salt ',
     ' vegetable oil plus salt ',
     ' lard plus salt'],
    'butter (unsalted)': ['shortening ', ' vegetable oil ', ' lard']}},
  'urls': {'source': 'http://simple-nourished-living.com/archives/729/easy-blonde-brownies-recipe/',
   'img': 'https://spoonacular.com/recipeImages/525788-556x370.jpg'},
  'usedCount': 3,
  'usedIngs': ['egg', 'milk', 'vanilla']},
 'Skinny Low Fa

In [6]:
import json
with open('testout.json', 'w', encoding='utf-8') as f:
    json.dump(allr.to_json(), f)